# Final Programming1 Assignment

It is well known that an excessive alcohol consumption can cause the increased risk of health problems. However, alcoholic beverages are highly available in any european country and the majority of people can not imagine a party without an alcohol. A lot of people believe that moderate alcohol intake can actually help them to release stress and feel more relaxed. But others avoid alcohol consumption not to harm their physical and psychological health. This project aims to reveal the truth - **is there a relation between the frequency of alcohol consumption and current depressive symptoms in Europe?**

#### About the data

The data is generated by the statistical office of the European Union - Eurostat. Please load the dataset of [alcohol consumption](https://ec.europa.eu/eurostat/databrowser/view/HLTH_EHIS_AL1E$DV_1142/default/table?lang=en&category=yth.yth_health) in csv format.

Please also load the dataset of [depressive symptoms](https://ec.europa.eu/eurostat/databrowser/view/HLTH_EHIS_MH1E$DV_463/default/table?lang=en&category=qol.qol_hlt.qol_hlt_st) in csv format. The general coverage of the survey is the population aged 15 or over living inside European Union. The surveys were conducted in 2014 and 2019 separately for males and females with different educational levels. The data is given in percentage of population per country.

##### Special flag value (in both datasets):
If OBS_FLAG is set to "u" it means that the data has low reliability. It was decided to exclude such data from the statistical tests.

##### Frequency (column called 'frequenc') of alcohol consumption:
 - DAY - every day
 - WEEK - every week
 - MTH - every month
 - LT1M - less than once a month
 - NM12 - not in the last 12 months
 - NVR - never
 - NVR_NM12 - never or not in the last 12 months

##### International Standard Classification of Education (column called isced11):
 - TOTAL - all isced 2011 levels
 - ED0-2 - less than primary, primary and lower secondary education
 - ED3-4 - upper secondary and post-secondary non-tertiary education
 - ED5-8 - Tertiary education (levels 5-8)

##### Age class:
 - Y15-19 - from 15 to 19
 - Y15-24 - from 15 to 24
 - (other age classes with the same logic for naming)
 - Y_GE18 - 18 years or over
 - Y_GE65 - 65 years or over
 - Y_GE75 - 75 years or over
 - TOTAL

##### Health problems:
 - DPR - Depressive symptoms
 - DPR_MJR - Major depressive symptoms
 - DPR_OTH - Other depressive symptoms


In [ ]:
import numpy as np
import yaml
import pandas as pd

def get_config():
    with open("config.yaml", 'r') as stream:
        return yaml.safe_load(stream)

config = get_config()

def read_csv(yaml_key):
    filepath = (config[yaml_key])
    if not filepath.endswith('.csv'):
        raise Exception
    df = pd.read_csv(filepath)
    return df

alcohol_df = read_csv('alcohol_consumption')
depression_df = read_csv('depressive_symptoms')
alcohol_df.head()

In [ ]:
depression_df.head()

##### Clear the data from the redundant columns
If the column contains one unique value within the whole data this column is considered redundant.

In [ ]:
def checkTheAmountOfUniqueValues(df, columnNames):
    res = []
    for columnName in columnNames:
        res.append(len(df[columnName].unique()))
    return res

# check that values are redundant (only one unique for all data)
redundant_column_names = ["DATAFLOW", "LAST UPDATE", "freq", "unit"]
redundant_alcohol = checkTheAmountOfUniqueValues(alcohol_df, redundant_column_names)
redundant_depression = checkTheAmountOfUniqueValues(depression_df, redundant_column_names)
print(f'Amount of unique values for {redundant_column_names} in alcohol consumption table: {redundant_alcohol}')
print(f'Amount of unique values for {redundant_column_names} in depressive symptoms table: {redundant_depression}')

# get rid of redundant columns
alcohol_df.drop(columns =redundant_column_names, inplace = True)
depression_df.drop(columns =redundant_column_names, inplace = True)

alcohol_df.head()

#### Clear data from NaN values
Check if there are Nan values and remove them from the data.

In [ ]:
# drop nan values
check_nan_values_alcohol = alcohol_df['OBS_VALUE'].isna().values.any()
check_nan_values_depr_sympts = depression_df['OBS_VALUE'].isna().values.any()
print(f'There are NaN values in alcohol consumption table - {check_nan_values_alcohol} and in depressive symptoms table - {check_nan_values_depr_sympts}')
alcohol_df.dropna(subset=['OBS_VALUE'], inplace=True)
depression_df.dropna(subset=['OBS_VALUE'], inplace=True)

#### Inspect the data
Check what data we are working with - what categories for location, age, educational level and time period we have. Compare it within two tables and prepare for being merged.

In [ ]:
def filter_rows_by_values(df, columnName, values):
    return df[df[columnName].isin(values)]

locations_in_total_al = alcohol_df["geo"].unique()
locations_in_total_dep = depression_df["geo"].unique()
print(f'Possible locations: {locations_in_total_al}, len: {len(locations_in_total_al)}')
print(f'Possible locations are the same?: {np.array_equal(locations_in_total_al, locations_in_total_dep)}')

# let's use only the statistics for separate countries
countries = [country for country in locations_in_total_al if len(country ) == 2]
alcohol_df = filter_rows_by_values(alcohol_df, "geo", countries)
depression_df = filter_rows_by_values(depression_df, "geo", countries)

# let's check possible age groups
age_in_total_al = alcohol_df["age"].unique()
age_in_total_dep = depression_df["age"].unique()
print(f'Possible age groups for alcohol consumption table: {age_in_total_al}, len: {len(age_in_total_al)}')
print(f'Possible age groups for depressive symptoms table: {age_in_total_dep}, len: {len(age_in_total_dep)}')
# 'Y25-64' is missing in the depressive symptoms table
alcohol_df.drop(alcohol_df[alcohol_df['age'] == 'Y25-64'].index, inplace=True)

# let's check possible education levels
edu_in_total_al = alcohol_df["isced11"].unique()
edu_in_total_dep = depression_df["isced11"].unique()
print(f'Possible education levels are the same?: {np.array_equal(edu_in_total_al, edu_in_total_dep)}')
print(f'Possible education levels: {edu_in_total_al}, len: {len(edu_in_total_al)}')

time_in_total_al = alcohol_df["TIME_PERIOD"].unique()
time_in_total_dep = depression_df["TIME_PERIOD"].unique()
print(f'Possible time periods are the same?: {np.array_equal(time_in_total_al, time_in_total_dep)}')
print(f'Possible time periods: {time_in_total_al}, len: {len(time_in_total_al)}')

#### Clear the data from low reliability values
As was mentioned above, there is the flag value - 'u' - which means that the data has low reliability. It was decided to eliminate such values from the data to get the most accurate results conducting statistical tests.

In [ ]:
# get rid of the data with low reliability
low_reliability_values_alc = alcohol_df[alcohol_df['OBS_FLAG'] == 'u']
low_reliability_values_dep = depression_df[depression_df['OBS_FLAG'] == 'u']
print(f'There are {len(low_reliability_values_alc)} low reliability values out of {len(alcohol_df)}')
print(f'There are {len(low_reliability_values_dep)} low reliability values out of {len(depression_df)}')

alcohol_df.drop(alcohol_df[alcohol_df['OBS_FLAG'] == 'u'].index, inplace=True)
depression_df.drop(depression_df[depression_df['OBS_FLAG'] == 'u'].index, inplace=True)

# we don't need the flags anymore
alcohol_df.drop("OBS_FLAG", axis='columns', inplace=True)
depression_df.drop("OBS_FLAG", axis='columns', inplace=True)

#### Data Wrangling: set up the dataframe
Combine data to create a dataframe with values of interest for the statistical analysis. Check the categories for the health problems (depressive symptoms), frequency of alcohol consumption and sex. To answer the research question we don't need to separate people by age, sex or educational level, so we fetch the data with 'TOTAL' values for these categories. There is no total value for the health problems or alcohol consumption frequency, therefore we use most common depressive symptoms and the daily alcohol consumption for these categories respectively.

In [ ]:
statistics = alcohol_df.merge(depression_df, how='inner', on=['geo', 'sex', 'age', 'TIME_PERIOD', 'isced11'], suffixes=('_alc', '_dep'))

health_problems = depression_df["hlth_pb"].unique()
print(f'Possible health problems: {health_problems}, len: {len(health_problems)}')

frequencies_al = alcohol_df["frequenc"].unique()
print(f'Possible frequencies: {frequencies_al}, len: {len(frequencies_al)}')

frequencies_al = alcohol_df["sex"].unique()
print(f'Possible sex: {frequencies_al}, len: {len(frequencies_al)}')

def getTable(df, dep_symptoms, age, sex, time_period, frequency, edu_level):
    return df[(df['hlth_pb'] == dep_symptoms) & (df['age'] == age) & (df['sex'] == sex) & (df['TIME_PERIOD'] == time_period) & (df['frequenc'] == frequency) & (df['isced11'] == edu_level)]

res_2014 = getTable(statistics, 'DPR', 'TOTAL', 'T', 2014, 'DAY', 'TOTAL')
res_2014.head()

#### Plot the data
Here we plot the data for both 2014 and 2019 to represent the correlation between the daily alcohol consumption and the development of depressive symptoms for the countries inside EU. The data is given in percentage of population per certain country. The stacked bar plot was chosen for data illustration to show the numerical values across two datasets at the same time in a comprehensive way.


In [ ]:
import matplotlib.pyplot as plt
def showPlot(df, year):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    countries = df['geo']
    y1, y2 = df['OBS_VALUE_alc'], df['OBS_VALUE_dep']
    ax.bar(countries, y1, color = 'b', alpha=0.8)
    ax.bar(countries, y2, bottom=y1, color = 'r', alpha=0.8)
    plt.xlabel("Countries")
    plt.ylabel("Percentage of population")
    plt.legend(["Alcohol consumption", "Depressive symptoms"])
    plt.title(f"Correlation between alcohol consumption on daily basis and the depressive symptoms development in {year} inside EU")
    plt.show()

showPlot(res_2014, 2014)

res_2019 = getTable(statistics, 'DPR', 'TOTAL', 'T', 2019, 'DAY', 'TOTAL')
showPlot(res_2019, 2019)

#### Conduct statistical analysis
Check for the data normality using Shapiro–Wilk test. Use a Levene’s & Bartlett’s Test of Equality (Homogeneity) of Variance to test equal variance. Conduct the paired samples t-test for the alcohol consumption and depressive symptoms values for both years.

In [ ]:
from scipy import stats
from scipy.stats import levene

def conductTests(df):
    y1, y2 = df['OBS_VALUE_alc'], df['OBS_VALUE_dep']
    # (for 2014) the p-value is 0.002773334039375186 which is less than the alpha(0.05) - the sample does not come from a normal distribution.
    print(stats.shapiro(y1))
    # (for 2014) the p-value is 0.15160074830055237 which is NOT less than the alpha(0.05) - the sample comes from a normal distribution.
    print(stats.shapiro(y2))
    # (for 2014) p-value (0.00088) is less than 0.05, so we can't conduct the two-sample t-test because the variance differs
    print("Levene test(alcohol_consumption~depressive_symptoms):\n", levene(y1, y2))
    # let's conduct paired samples t-test: (for 2014) p-value (0.76) is not less than 0.05, so there is no significant interaction effect between the alcohol consumption and the depressive symptoms
    print("Paired samples t-test results:\n", stats.ttest_rel(y1, y2))

print('\n2014 YEAR:')
conductTests(res_2014)
print('\n2019 YEAR:')
conductTests(res_2019)

#### Conduct one-way ANOVA test
Let's test if the educational level has significant effect on the development of depressive symptoms (the data only for 2014 was used).

In [ ]:
# one-way ANOVA: percentage of people having depressive symptoms - dependent vars, categories of education level - categorical vars
# get the table with ALL possible education levels
data_anova1 = statistics[(statistics['hlth_pb'] == 'DPR') & (statistics['age'] == 'TOTAL') & (statistics['sex'] == 'T') & (statistics['TIME_PERIOD'] == 2014) & (statistics['frequenc'] == 'DAY')]

# get rid of the 'TOTAL' education level (let's work only with three categories: 'ED0-2' 'ED3_4' 'ED5-8')
data_anova1 = data_anova1.drop(data_anova1[data_anova1['isced11'] == 'TOTAL'].index)

groups_frame_anova1 = pd.DataFrame({"Dep_symptoms":data_anova1['OBS_VALUE_dep'],"Edu_level":data_anova1['isced11']})
groups_by_category_of_edu_level = groups_frame_anova1.groupby("Edu_level")

edo2_group = groups_by_category_of_edu_level.get_group('ED0-2')['Dep_symptoms']
ed34_group = groups_by_category_of_edu_level.get_group('ED3_4')['Dep_symptoms']
ed58_group = groups_by_category_of_edu_level.get_group('ED5-8')['Dep_symptoms']

# P(1.05e-13) << 0.05. It means that the educational level has significant effect on the development of depressive symptoms
print("one-way ANOVA:\n", stats.f_oneway(edo2_group, ed34_group, ed58_group), "\n")

#### Conduct two-way ANOVA test:
Let's test if the educational level and the frequency of alcohol consumption cause an effect on the percentage of population consuming alcohol (the data only for 2014 was used).

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# two-way ANOVA: percentage of people consuming alcohol - dependent vars, categories of consumption frequency AND educational levels - categorical vars
# get the table with ALL possible consumption frequencies and educational levels
data = statistics[(statistics['hlth_pb'] == 'DPR') & (statistics['age'] == 'TOTAL') & (statistics['sex'] == 'T') & (statistics['TIME_PERIOD'] == 2014)]

# consumption frequencies: 'DAY' 'LT1M' 'MTH' 'NM12' 'NVR' 'NVR_NM12' 'WEEK'
# get rid of the 'TOTAL' education level (let's work only with three categories: 'ED0-2' 'ED3_4' 'ED5-8')
data = data.drop(data[data['isced11'] == 'TOTAL'].index)

groups_frame = pd.DataFrame({"Alc_consumption":data['OBS_VALUE_alc'],
                             "Edu_level":data['isced11'],
                             'Frequency': data['frequenc']})

model = ols('Alc_consumption ~ C(Edu_level) + C(Frequency) + C(Edu_level):C(Frequency)', data=groups_frame).fit()

# P for C(Edu_level):C(Frequency) << 0.05 - there is significant interaction effect between educational level and the frequency of alcohol consumption
# Both factors do have statistically significant effect on the amount of people consuming alcohol (their p-values << 0.05)
print("two-way ANOVA:\n", sm.stats.anova_lm(model, type=2), "\n")

#### Create an interactive visualisation of data
Statistical analysis was conducted only for the small sample of data (with 'TOTAL' values for age, sex, educational level). The interactive plot lets the user see the correlation between alcohol consumption and depressive symptoms development in european countries for different sex, age, educational level and alcohol consumption frequency.

Legends:
 - AC - alcohol consumption
 - DS - depressive symptoms

In [ ]:
from bokeh.models import FactorRange
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
from bokeh.resources import INLINE
output_notebook(INLINE)

In [ ]:
def buildPlot(source):
    countries = source['geo'].values
    values = ['AC', 'DS']
    data = {'geo' : countries,
            'AC'   : source['OBS_VALUE_alc'].values,
            'DS'   : source['OBS_VALUE_dep'].values
            }

    x = [ (country, value) for country in countries for value in values ]
    counts = sum(zip(data['AC'], data['DS']), ())
    legends = [v[1] for v in x ]

    source_column = ColumnDataSource(data=dict(x=x, counts=counts, legends = legends))

    p = figure(x_range=FactorRange(*x), title="Correlation between alcohol consumption and depressive symptoms development", plot_width=1100, plot_height=600)

    p.vbar(x='x', top='counts', width=1, source=source_column, line_color="white", legend_field="legends",
           fill_color=factor_cmap('x', palette=Spectral6, factors=values, start=1, end=2))

    p.y_range.start = 0
    p.x_range.group_padding = 3.0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None
    p.yaxis.axis_label = "Population percentage"
    return p

def GetPlot(symptoms, age, sex, year, freq, edu_level):
    source = getTable(statistics, symptoms, age, sex, year, freq, edu_level)
    return buildPlot(source)

In [ ]:
import panel as pn
from panel import widgets
pn.extension()

frequencies = ['DAY', 'LT1M', 'MTH', 'NM12', 'NVR', 'NVR_NM12', 'WEEK']
select_freq = widgets.Select(
    name='Select alcohol consumption frequency',
    options=frequencies,
    size=1)

time_period = [2014, 2019]
select_time = widgets.Select(
    name='Select time period',
    options=time_period,
    size=1)

depr_symptoms = ['DPR', 'DPR_MJR', 'DPR_OTH']
select_symptoms = widgets.Select(
    name='Select depressive symptoms',
    options=depr_symptoms,
    size=1)

sex = ['F', 'M', 'T']
select_sex = widgets.Select(
    name='Select sex',
    options=sex,
    size=1)

age = list(statistics['age'].unique())
select_age = widgets.Select(
    name='Select age',
    options=age,
    size=1)

edu_level = list(statistics['isced11'].unique())
select_edu_level = widgets.Select(
    name='Select educational level',
    options=edu_level,
    size=1)

stacked_plot = pn.bind(GetPlot, symptoms=select_symptoms, age=select_age, sex=select_sex, year=select_time, freq=select_freq, edu_level=select_edu_level)
pn.Column(pn.Row(select_freq, select_time, select_symptoms), pn.Row(select_sex, select_age, select_edu_level), stacked_plot)

#### Conclusion
The interactive plot showed that three countries which have the largest percentage of population drinking alcohol on daily basis are PT (Portugal, almost 40%!), ES (Spain) and IT (Italy) in 2014. Other three countries which population never consumes alcohol are CY (Cyprus, about 45%), HR(Croatia) and IT (Italy) in 2014.
Plot doesn't show any noticeable correlation between the development of depressive symptoms and alcohol consumption (as well as the t-test). It also proves that there is the correlation between the educational level and the development of depressive symptoms (the one-way ANOVA test) - the higher the education level the smaller amount of population struggles with depressive symptoms. It also can be seen that most people consume alcohol several times per month or monthly.

